In [1]:
import os
import emcee

import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
from pyazr import azure2

from multiprocess import Pool

# Restrict processes to one thread only
os.environ['OMP_NUM_THREADS'] = '1'

# emcee variables
nsteps = 10000 # How many steps should each walker take?
nthin  = 1     # How often should the walker save a step?
nprocs = 20    # How many Python processes do you want to allocate?

# Define the data labels (in AZURE2 order)
labels = ["Meyer et al. (1976) - 84.3 deg",
          "Meyer et al. (1976) - 114.5 deg",
          "Meyer et al. (1976) - 144.1 deg", 
          "LUNA HPGe (2023)", 
          "LUNA BGO (2023)",
          "Felsenkeller (2023)",
          "ATOMKI (2023)",
          "Notre Dame (2023) - 0 deg",
          "Notre Dame (2023) - 55 deg",
          "Burtebaev et al. (2008)",
          "Lamb et al. (1957)",
          "Bailey et al. (1950)",
          "Vogl et al. (1963)",
          "Rolfs et al. (1974) - 0 deg",
          "Rolfs et al. (1974) - 90 deg"]

# Normalization parameter map : { index : (value, error) } (in AZURE2 order)
norms = { 0:  (1, 0.05),   1:  (1, 0.05),   2: (1, 0.05),  
          3:  (1, 0.069),  4:  (1, 0.079),  5: (1, 0.1),  
          6:  (1, 0.1),    7:  (1, 0.1),    8: (1, 0.1),  
          9:  (1, 0.1),    10: (1, 0.0),   11: (1, 0.0), 
          12: (1, 0.0),    13: (1, 0.0),   14: (1, 0.0) }

# Define the parameters prior distributions
priors = [
    stats.norm(1.63,0.12),

    stats.uniform(2.30, 0.10), stats.uniform(0, 100000), stats.uniform(-10, 20),
    stats.uniform(-1000000,2000000),

    stats.uniform(3.45, 0.10), stats.uniform(0, 100000), stats.uniform(-10, 20), stats.uniform(-10, 20),
    stats.uniform(-1000000,2000000), stats.uniform(-1000000,2000000),

    stats.uniform(3.50, 0.10), stats.uniform(0, 100000),
    
    stats.norm(1.0,0.05),
    stats.norm(1.0,0.05),
    stats.norm(1.0,0.05),
    stats.norm(1.0,0.069),
    stats.norm(1.0,0.079),
    stats.norm(1.0,0.10),
    stats.norm(1.0,0.10),
    stats.norm(1.0,0.10),
    stats.norm(1.0,0.10),
    stats.norm(1.0,0.10),
    stats.uniform(0, 100),
    stats.uniform(0, 100),
    stats.uniform(0, 100),
    stats.uniform(0, 100),
    stats.uniform(0, 100)
]

In [2]:
# We read the .azr file and set the external capture file to speed up the calculation
azr = azure2('12c_pg.azr', nprocs=nprocs)

In [3]:
# We get the initial values from AZURE2
theta0 = azr.params
ntheta = len(theta0)

# Now we add the normalizations
for norm in norms.items( ):
    theta0 = np.concatenate( (theta0, [norm[1][0]]) )

# We'll read the data from the output file since it's already in the center-of-mass frame
y = azr.cross
yerr = azr.cross_err
ndata = sum( len(segment) for segment in y )

# Number of Walkers = 2 x Number of Parameters
nw = 2 * ntheta

In [ ]:
# Prior log probability
def lnPi( theta ):
    return np.sum([pi.logpdf(t) for (pi, t) in zip(priors, theta)])

# Log likelihood
def lnl( theta, proc=0 ):
    mu = azr.calculate( theta[:ntheta], proc=proc )
    lnl = np.sum( -0.5 * np.log(2 * np.pi* pow(yerr, 2) ) - 0.5 * pow((y - mu) / yerr, 2) )
    return lnl

# Posterior log probability
def lnP( theta ):
    lnpi = lnPi( theta )
    if not np.isfinite( lnpi ): return -np.inf
    return lnl( theta ) + lnpi

In [4]:
# Preparing initial parameters for the walkers
p0 = np.zeros((nw, ntheta))
mean, std = np.loadtxt('minuit/best_lnl.txt')[:,0], np.loadtxt('minuit/best_lnl.txt')[:,1]
for i in range(nw): p0[i, :] = stats.norm(mean, std).rvs()

In [5]:
backend = emcee.backends.HDFBackend('results/mcmc/samples.h5') 
backend.reset(nw, ntheta)

with Pool(processes=nprocs) as pool:
    sampler = emcee.EnsembleSampler( nw, ntheta, lnP, pool=pool, backend=backend ) 
    state = sampler.run_mcmc( p0, nsteps, thin_by=nthin, progress=True, tune=True )

100%|██████████| 1000/1000 [6:20:17<00:00, 22.82s/it]   
